# Q2 (b): Dantzig-Wolfe

In [302]:
import numpy as np
from numpy import random
from gurobipy import *

In [376]:
# ITERATION 1
# Solving the restricted master problem
RMP = Model('RMP')
c = np.array([10,1,1,1,10,1,1,1,10,15,15,15])
b = np.array([1,1,1])
A = np.array([[1,0,0,1,0,0,1,0,0,0,0,0],
             [0,1,0,0,1,0,0,1,0,0,0,0],
             [0,0,1,0,0,1,0,0,1,0,0,0]])

v = []

v.append(np.array([1,0,0,0,1,0,0,0,1,1,1,1]))
# v.append(np.array([0,1,0,1,0,0,0,0,1,1,1,1]))
# v.append(np.array([0,0,0,0,0,0,1,1,1,0,0,1]))
# v.append(np.array([0,0,0,1,1,1,0,0,0,0,1,0]))
# Define Variables
alpha = {}

for i in range(len(v)):
    alpha[i] = RMP.addVar(name='alpha'+str(i))

# Define Objective
RMP.setObjective(quicksum(alpha[i]*quicksum(c[j]*v[i][j] for j in range(len(v[i]))) for i in range(len(v))))


# Define Constraints
Constr = {} # indicator constraints
Constr['alpha'] = RMP.addConstr(quicksum(alpha[i] for i in alpha.keys()),GRB.EQUAL,1,name='Constr_alpha')
for row in range(len(b)):
    Constr[row] = RMP.addConstr(quicksum(alpha[i]*quicksum(A[row][j]*v[i][j] for j in range(len(v[i]))) for i in range(len(v))),
                                GRB.EQUAL,1,name='Constr'+str(row))
#     =======================================================================================


#Add constraints and variables to model
RMP.update()
# Initialize Model and Solver Settings NOTE: DO NOT EDIT THE SETTINGS IN THIS BLOCK UNLESS OTHERWISE NOTED IN THE EXERCISE
RMP.setParam('TimeLimit', 900)
RMP.setParam('DualReductions',0)
# ufl.setParam('NodeLimit', 10) # 


RMP.modelSense = GRB.MINIMIZE
RMP.update()
# Optimize Model (you should see output when running this cell)
RMP.optimize()

Changed value of parameter TimeLimit to 900.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter DualReductions to 0
   Prev: 1  Min: 0  Max: 1  Default: 1
Optimize a model with 4 rows, 1 columns and 4 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e+01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 4 rows and 1 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.5000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  7.500000000e+01


In [377]:
alpha

{0: <gurobi.Var alpha0 (value 1.0)>}

In [372]:
# Solving the dual
dual = Model('dual')

# Define Variables
pi = {}
for ind_i in range(3):
    pi[ind_i] = dual.addVar(lb=-GRB.INFINITY, name='pi'+str(ind_i))
pi_0 = dual.addVar(lb=-GRB.INFINITY, name='pi_0')

# Define Objective
dual.setObjective(pi_0+pi[0]+pi[1]+pi[2])

rhs = []
for i in range(len(v)):
    rhs.append(sum(c[j]*v[i][j] for j in range(len(v[i]))))

# Define Constraints
Constr = {} # indicator constraints

for i in range(len(v)):
    Constr[i] = dual.addConstr(quicksum(pi[row]*quicksum(A[row][j]*v[i][j] for j in range(len(v[i]))) for row in range(len(b)))+pi_0,
                                '<=',rhs[i],name='Constr'+str(i))

#     =======================================================================================


#Add constraints and variables to model
dual.update()
# Initialize Model and Solver Settings NOTE: DO NOT EDIT THE SETTINGS IN THIS BLOCK UNLESS OTHERWISE NOTED IN THE EXERCISE
dual.setParam('TimeLimit', 900)
dual.setParam('DualReductions',0)
# ufl.setParam('NodeLimit', 10) # 


dual.modelSense = GRB.MAXIMIZE
dual.update()
# Optimize Model (you should see output when running this cell)
dual.optimize()

Changed value of parameter TimeLimit to 900.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter DualReductions to 0
   Prev: 1  Min: 0  Max: 1  Default: 1
Optimize a model with 1 rows, 4 columns and 4 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 3e+01]
Presolve time: 0.00s
Presolved: 1 rows, 4 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0      handle free variables                          0s
       1    2.7000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds
Optimal objective  2.700000000e+01


In [373]:
pi

{0: <gurobi.Var pi0 (value 27.0)>,
 1: <gurobi.Var pi1 (value 0.0)>,
 2: <gurobi.Var pi2 (value 0.0)>}

In [374]:
pi_0

<gurobi.Var pi_0 (value 0.0)>

In [375]:
# Calculate c-pi_j*A_j
for i in range(3):
    print(c-pi[i].X*A[i])

[-17.   1.   1. -26.  10.   1. -26.   1.  10.  15.  15.  15.]
[10.  1.  1.  1. 10.  1.  1.  1. 10. 15. 15. 15.]
[10.  1.  1.  1. 10.  1.  1.  1. 10. 15. 15. 15.]


In [333]:
v

[array([0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1])]

In [324]:
a = c-pi[0].X*A[0]
b = np.where(a>0)[0]
k = [np.power(2,len(b))/2]
while k[-1] > 1:
    k.append(k[-1]/2)
for i in range(np.power(2,len(b))):
    d = np.array([0,0,0,0,0,0,0,0,0,0,0,0])
    for j in range(len(b)):
        if np.floor(i/k[j]) % 2 == 1:
            d[b[j]] = 1
    print(d)
    v.append(d)

[0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 0 0 0 1 0]
[0 0 0 0 0 0 0 0 0 0 1 1]
[0 0 0 0 0 0 0 0 0 1 0 0]
[0 0 0 0 0 0 0 0 0 1 0 1]
[0 0 0 0 0 0 0 0 0 1 1 0]
[0 0 0 0 0 0 0 0 0 1 1 1]
[0 0 0 0 0 0 0 0 1 0 0 0]
[0 0 0 0 0 0 0 0 1 0 0 1]
[0 0 0 0 0 0 0 0 1 0 1 0]
[0 0 0 0 0 0 0 0 1 0 1 1]
[0 0 0 0 0 0 0 0 1 1 0 0]
[0 0 0 0 0 0 0 0 1 1 0 1]
[0 0 0 0 0 0 0 0 1 1 1 0]
[0 0 0 0 0 0 0 0 1 1 1 1]
[0 0 0 0 0 0 0 1 0 0 0 0]
[0 0 0 0 0 0 0 1 0 0 0 1]
[0 0 0 0 0 0 0 1 0 0 1 0]
[0 0 0 0 0 0 0 1 0 0 1 1]
[0 0 0 0 0 0 0 1 0 1 0 0]
[0 0 0 0 0 0 0 1 0 1 0 1]
[0 0 0 0 0 0 0 1 0 1 1 0]
[0 0 0 0 0 0 0 1 0 1 1 1]
[0 0 0 0 0 0 0 1 1 0 0 0]
[0 0 0 0 0 0 0 1 1 0 0 1]
[0 0 0 0 0 0 0 1 1 0 1 0]
[0 0 0 0 0 0 0 1 1 0 1 1]
[0 0 0 0 0 0 0 1 1 1 0 0]
[0 0 0 0 0 0 0 1 1 1 0 1]
[0 0 0 0 0 0 0 1 1 1 1 0]
[0 0 0 0 0 0 0 1 1 1 1 1]
[0 0 0 0 0 0 1 0 0 0 0 0]
[0 0 0 0 0 0 1 0 0 0 0 1]
[0 0 0 0 0 0 1 0 0 0 1 0]
[0 0 0 0 0 0 1 0 0 0 1 1]
[0 0 0 0 0 0 1 0 0 1 0 0]
[0 0 0 0 0 0 1 0 0 1 0 1]
[0 0 0 0 0 0

In [334]:

v.append(np.array([0,0,0,0,0,0,1,1,1,0,0,1]))

In [335]:
# ITERATION 2
# Solving the restricted master problem
RMP = Model('RMP')
c = np.array([10,1,1,1,10,1,1,1,10,15,15,15])
b = np.array([1,1,1])
A = np.array([[1,1,1,0,0,0,0,0,0,0,0,0],
             [0,0,0,1,1,1,0,0,0,0,0,0],
             [0,0,0,0,0,0,1,1,1,0,0,0]])

# Define Variables
alpha = {}

for i in range(len(v)):
    alpha[i] = RMP.addVar(name='alpha'+str(i))

# Define Objective
RMP.setObjective(quicksum(alpha[i]*quicksum(c[j]*v[i][j] for j in range(len(v[i]))) for i in range(len(v))))


# Define Constraints
Constr = {} # indicator constraints
Constr['alpha'] = RMP.addConstr(quicksum(alpha[i] for i in alpha.keys()),GRB.EQUAL,1,name='Constr_alpha')
for row in range(len(b)):
    Constr[row] = RMP.addConstr(quicksum(alpha[i]*quicksum(A[row][j]*v[i][j] for j in range(len(v[i]))) for i in range(len(v))),
                                GRB.EQUAL,1,name='Constr'+str(row))
#     =======================================================================================


#Add constraints and variables to model
RMP.update()
# Initialize Model and Solver Settings NOTE: DO NOT EDIT THE SETTINGS IN THIS BLOCK UNLESS OTHERWISE NOTED IN THE EXERCISE
RMP.setParam('TimeLimit', 900)
RMP.setParam('DualReductions',0)
# ufl.setParam('NodeLimit', 10) # 


RMP.modelSense = GRB.MINIMIZE
RMP.update()
# Optimize Model (you should see output when running this cell)
RMP.optimize()

Changed value of parameter TimeLimit to 900.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter DualReductions to 0
   Prev: 1  Min: 0  Max: 1  Default: 1
Optimize a model with 4 rows, 2 columns and 6 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [3e+01, 6e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 4 rows and 2 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.7000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds
Optimal objective  5.700000000e+01


In [336]:
alpha

{0: <gurobi.Var alpha0 (value 1.0)>, 1: <gurobi.Var alpha1 (value 0.0)>}

In [312]:
# Solving the dual
dual = Model('dual')

# Define Variables
pi = {}
for ind_i in range(3):
    pi[ind_i] = dual.addVar(lb=-GRB.INFINITY, name='pi'+str(ind_i))
pi_0 = dual.addVar(lb=-GRB.INFINITY, name='pi_0')

# Define Objective
dual.setObjective(pi_0+pi[0]+pi[1]+pi[2])

rhs = []
for i in range(len(v)):
    rhs.append(sum(c[j]*v[i][j] for j in range(len(v[i]))))

# Define Constraints
Constr = {} # indicator constraints

for i in range(len(v)):
    Constr[i] = dual.addConstr(quicksum(pi[row]*quicksum(A[row][j]*v[i][j] for j in range(len(v[i]))) for row in range(len(b)))+pi_0,
                                '<=',rhs[i],name='Constr'+str(i))

#     =======================================================================================


#Add constraints and variables to model
dual.update()
# Initialize Model and Solver Settings NOTE: DO NOT EDIT THE SETTINGS IN THIS BLOCK UNLESS OTHERWISE NOTED IN THE EXERCISE
dual.setParam('TimeLimit', 900)
dual.setParam('DualReductions',0)
# ufl.setParam('NodeLimit', 10) # 


dual.modelSense = GRB.MAXIMIZE
dual.update()
# Optimize Model (you should see output when running this cell)
dual.optimize()

Changed value of parameter TimeLimit to 900.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter DualReductions to 0
   Prev: 1  Min: 0  Max: 1  Default: 1
Optimize a model with 9 rows, 4 columns and 19 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 6e+01]
Presolve removed 5 rows and 0 columns
Presolve time: 0.00s
Presolved: 4 rows, 4 columns, 10 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0      handle free variables                          0s
       2    5.7000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds
Optimal objective  5.700000000e+01


In [313]:
pi

{0: <gurobi.Var pi0 (value 1.0)>,
 1: <gurobi.Var pi1 (value 56.0)>,
 2: <gurobi.Var pi2 (value 0.0)>}

In [314]:
pi_0

<gurobi.Var pi_0 (value 0.0)>

In [315]:
# Calculate c-pi_j*A_j
for i in range(3):
    print(c-pi[i].X*A[i])

[ 9.  0.  0.  1. 10.  1.  1.  1. 10. 15. 15. 15.]
[ 10.   1.   1. -55. -46. -55.   1.   1.  10.  15.  15.  15.]
[10.  1.  1.  1. 10.  1.  1.  1. 10. 15. 15. 15.]


In [298]:
a = c-pi[0].X*A[0]
b = np.where(a<=0)[0]
k = [np.power(2,len(b))/2]
while k[-1] > 1:
    k.append(k[-1]/2)
for i in range(np.power(2,len(b))):
    d = np.array([0,0,0,0,0,0,0,0,0,0,0,0])
    for j in range(len(b)):
        if np.floor(i/k[j]) % 2 == 1:
            d[b[j]] = 1
    print(d)
#     v.append(d)

[0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0 0 0 0 0]
[0 1 1 0 0 0 0 0 0 0 0 0]
[1 0 0 0 0 0 0 0 0 0 0 0]
[1 0 1 0 0 0 0 0 0 0 0 0]
[1 1 0 0 0 0 0 0 0 0 0 0]
[1 1 1 0 0 0 0 0 0 0 0 0]
